# Graph-Based Recommendation with NetworkX (3.x-safe)

Graphs are a natural way to think about recommendation problems because users, items, and their interactions already form edges of a network. In this short workshop we walk from raw interactions through several graph-based recommenders.

**Session duration:** ~45 minutes

**Learning objectives**

- Understand how a user-item bipartite graph supports recommendation tasks.
- Build an item-item projection and compute lightweight similarity scores.
- Compare graph-based strategies (co-occurrence, Jaccard, Adamic-Adar, Personalized PageRank) against a simple popularity baseline.
- Evaluate recommendation quality with leave-one-out metrics and interpret the results.

**Suggested timeline**

1. Warm-up and notebook tour (5 min)
2. Build the synthetic dataset and explore it (10 min)
3. Construct the graphs and discuss similarity heuristics (15 min)
4. Demo recommendations and hands-on exercises (10 min)
5. Quick evaluation recap and wrap-up (5 min)

> Tip for instructors: keep the code cells executed ahead of time if the audience is new to notebooks, but invite volunteers to run the "Your turn" prompts live.


## Setup

This notebook assumes Python 3.11+ and the packages pinned in `requirements.txt`.

- Run `pip install -r requirements.txt` once per environment.
- Restart the kernel if you upgrade `networkx` so cached results do not leak across runs.
- The imports below also configure pandas and matplotlib for inline plotting.

If you are running this during the live session, clear all outputs ahead of time so the audience can see results appear cell by cell.


In [ ]:
import itertools
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from IPython.display import display
from networkx.algorithms import bipartite

pd.options.display.float_format = "{:.3f}".format
try:
    plt.style.use("seaborn-v0_8")
except OSError:
    plt.style.use("seaborn-colorblind")

print("NetworkX:", nx.__version__)
print("Pandas:", pd.__version__)


## Dataset: synthetic user-item interactions

We fabricate a compact dataset that still exposes meaningful structure for discussion. Each user has interacted with four items. Items are grouped into loose thematic clusters so we can talk about "similar taste" later on.

We also attach a miniature catalog table that we can merge back into recommendations to make the results easier to narrate.


In [ ]:
# Users and items
users = [f"U{i}" for i in range(1, 13)]
item_catalog = pd.DataFrame(
    [
        ("I1", "Graph Theory"),
        ("I2", "Graph Theory"),
        ("I3", "Graph Theory"),
        ("I4", "Graph Theory"),
        ("I5", "Visualization"),
        ("I6", "Visualization"),
        ("I7", "Visualization"),
        ("I8", "Visualization"),
        ("I9", "Machine Learning"),
        ("I10", "Machine Learning"),
        ("I11", "Machine Learning"),
        ("I12", "Machine Learning"),
        ("I13", "Data Platforms"),
        ("I14", "Data Platforms"),
        ("I15", "Data Platforms"),
        ("I16", "Data Platforms"),
    ],
    columns=["item", "category"],
)
items = item_catalog["item"].tolist()

# Deterministic small dataset with overlapping tastes
interactions = {
    "U1": ["I1", "I2", "I3", "I7"],
    "U2": ["I2", "I3", "I4", "I8"],
    "U3": ["I1", "I3", "I5", "I9"],
    "U4": ["I4", "I5", "I6", "I10"],
    "U5": ["I2", "I6", "I7", "I11"],
    "U6": ["I1", "I5", "I7", "I12"],
    "U7": ["I8", "I9", "I10", "I13"],
    "U8": ["I3", "I9", "I11", "I14"],
    "U9": ["I6", "I10", "I12", "I15"],
    "U10": ["I2", "I11", "I13", "I16"],
    "U11": ["I4", "I8", "I12", "I14"],
    "U12": ["I5", "I9", "I15", "I16"],
}

# Convert to a DataFrame (implicit feedback: 1 per interaction)
df = pd.DataFrame(
    [(u, it, 1) for u, its in interactions.items() for it in its],
    columns=["user", "item", "weight"],
)

df.head()


### Inspect the interactions

We keep the dataset tiny on purpose so you can trace recommendations back to individual co-occurrences during the session.


In [ ]:
print(f"Users: {len(users)}  Items: {len(items)}  Interactions: {len(df)}")

user_counts = (
    df.groupby("user")["item"].count().rename("items_per_user").sort_values(ascending=False)
)
item_counts = (
    df.groupby("item")["user"].count().rename("users_per_item").sort_values(ascending=False)
)

print("Items per user (should stay balanced in this synthetic set):")
display(user_counts)

print("Users per item:")
display(item_counts)

print("Sample interactions with item metadata:")
display(
    df.merge(item_catalog, on="item", how="left")
    .sample(5, random_state=42)
    .sort_values(["user", "item"])
)


> **Your turn:** change the `interactions` dictionary to mirror your domain (movies, courses, internal documents) and re-run the next cells. Watch how the graph statistics react.


## Build a user-item bipartite graph

A bipartite graph separates users and items into two disjoint node sets. NetworkX lets us tag nodes with a `bipartite` attribute so we can keep the types apart.

Key talking points:

- Edges represent interactions (implicit feedback in this toy dataset).
- The degree of a user is the number of items they touched (and vice versa for items).
- We can already spot heavy users or popular items before computing any similarity.


In [ ]:
def build_bipartite_graph(df, user_col="user", item_col="item", weight_col="weight"):
    '''Return a NetworkX Graph with bipartite labels.'''
    G = nx.Graph(name="User-Item Graph")
    users_local = df[user_col].unique().tolist()
    items_local = df[item_col].unique().tolist()
    G.add_nodes_from(users_local, bipartite="user")
    G.add_nodes_from(items_local, bipartite="item")
    for _, row in df.iterrows():
        G.add_edge(row[user_col], row[item_col], weight=float(row.get(weight_col, 1.0)))
    return G


G = build_bipartite_graph(df)
user_nodes = [n for n, d in G.nodes(data=True) if d.get("bipartite") == "user"]
item_nodes = [n for n, d in G.nodes(data=True) if d.get("bipartite") == "item"]

density = bipartite.density(G, user_nodes)

print(G)
print(f"Users: {len(user_nodes)}  Items: {len(item_nodes)}  Edges: {G.number_of_edges()}")
print(f"Bipartite density: {density:.3f}")

avg_user_degree = sum(dict(G.degree(user_nodes)).values()) / len(user_nodes)
avg_item_degree = sum(dict(G.degree(item_nodes)).values()) / len(item_nodes)
print(f"Avg items per user: {avg_user_degree:.2f}")
print(f"Avg users per item: {avg_item_degree:.2f}")

example_user = user_nodes[0]
print(f"Example user {example_user} has items: {sorted(G.neighbors(example_user))}")


In [ ]:
user_degree = pd.Series(dict(G.degree(user_nodes)), name="degree").sort_values(
    ascending=False
)
item_degree = pd.Series(dict(G.degree(item_nodes)), name="degree").sort_values(
    ascending=False
)

print("Top users by degree:")
display(user_degree.head(5).to_frame())

print("Top items by degree:")
display(item_degree.head(5).to_frame())


## Item-item projection (co-occurrence)

Projecting the bipartite graph onto the item set links two items when they share at least one user. The edge weight counts the number of shared users (co-occurrence frequency).

This projection is the raw material for item-to-item recommenders such as Amazon's "customers who bought X also bought Y".


In [ ]:
G_item = bipartite.weighted_projected_graph(G, item_nodes)

print(G_item)
print(f"Projection edges: {G_item.number_of_edges()}")

edges_sorted = sorted(
    G_item.edges(data=True), key=lambda e: e[2].get("weight", 0.0), reverse=True
)[:10]

cooccurrence_table = pd.DataFrame(
    [(a, b, data.get("weight", 0.0)) for a, b, data in edges_sorted],
    columns=["item_a", "item_b", "shared_users"],
)

cooccurrence_table


## Item-item similarity via Jaccard and Adamic-Adar

Co-occurrence counts can be biased toward very popular items. Normalized similarity scores help rebalance the influence of hubs:

- **Jaccard** divides shared users by the union of users who touched each item.
- **Adamic-Adar** down-weights common neighbors that are themselves very popular.

We pre-compute these scores for every item pair so we can reuse them across users.


In [ ]:
from networkx.algorithms.link_prediction import adamic_adar_index, jaccard_coefficient

pairs = list(itertools.combinations(item_nodes, 2))

jacc = {tuple(sorted((a, b))): score for a, b, score in jaccard_coefficient(G, pairs)}

aa = {tuple(sorted((a, b))): score for a, b, score in adamic_adar_index(G, pairs)}

similarity_view = (
    pd.DataFrame(
        [
            (a, b, jacc[tuple(sorted((a, b)))], aa[tuple(sorted((a, b)))])
            for a, b in pairs
        ],
        columns=["item_a", "item_b", "jaccard", "adamic_adar"],
    )
    .sort_values(["jaccard", "adamic_adar"], ascending=False)
    .head(10)
)

similarity_view


## Recommender functions

We will compare four simple strategies:

1. **Popularity**: always recommend the most interacted items (non-graph baseline).
2. **Co-occurrence**: sum item-item edge weights that connect to the user's history.
3. **Jaccard**: use normalized overlap with existing items.
4. **Adamic-Adar**: reward rare overlaps more than frequent ones.
5. **Personalized PageRank**: random walks biased toward the user node.

Feel free to experiment with the aggregation strategies or add your own heuristics (for example, weighting recent interactions more heavily).


In [ ]:
def user_items(G, user):
    """Return the set of items a user has interacted with."""
    return {nbr for nbr in G.neighbors(user) if G.nodes[nbr].get("bipartite") == "item"}


def candidate_items(G, user):
    """Items the user has not interacted with yet."""
    owned = user_items(G, user)
    items_all = {n for n, d in G.nodes(data=True) if d.get("bipartite") == "item"}
    return list(items_all - owned)


def recommend_popularity(
    df, user, topk=10, item_col="item", user_col="user", weight_col="weight"
):
    """Rank items by global popularity and filter out items already seen by the user."""
    owned = set(df[df[user_col] == user][item_col].tolist())
    popularity = df.groupby(item_col)[weight_col].sum().sort_values(ascending=False)
    recs = popularity.drop(labels=list(owned), errors="ignore").head(topk)
    return recs.reset_index().rename(columns={item_col: "item", weight_col: "score"})


def recommend_cooccurrence(G, G_item, user, topk=10):
    """Score candidates by summing item-item co-occurrence weights."""
    owned = user_items(G, user)
    cand = candidate_items(G, user)
    scores = {}
    for c in cand:
        total = 0.0
        for o in owned:
            weight = (
                G_item.get_edge_data(c, o, default={"weight": 0.0})["weight"]
                if G_item.has_node(c) and G_item.has_node(o)
                else 0.0
            )
            total += float(weight)
        scores[c] = total
    recs = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:topk]
    return pd.DataFrame(recs, columns=["item", "score"])


def recommend_jaccard(G, user, jacc, agg="mean", topk=10):
    """Score candidates by aggregating Jaccard scores to the user's items."""
    owned = list(user_items(G, user))
    cand = candidate_items(G, user)
    scores = {}
    for c in cand:
        vals = [jacc.get(tuple(sorted((c, o))), 0.0) for o in owned]
        if not vals:
            scores[c] = 0.0
            continue
        scores[c] = max(vals) if agg == "max" else sum(vals) / len(vals)
    recs = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:topk]
    return pd.DataFrame(recs, columns=["item", "score"])


def recommend_adamic_adar(G, user, aa, agg="sum", topk=10):
    """Score candidates by aggregating Adamic-Adar scores."""
    owned = list(user_items(G, user))
    cand = candidate_items(G, user)
    scores = {}
    for c in cand:
        vals = [aa.get(tuple(sorted((c, o))), 0.0) for o in owned]
        if not vals:
            scores[c] = 0.0
            continue
        if agg == "mean":
            scores[c] = sum(vals) / len(vals)
        else:
            scores[c] = sum(vals)
    recs = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:topk]
    return pd.DataFrame(recs, columns=["item", "score"])


def recommend_ppr(G, user, topk=10, alpha=0.85):
    """Personalized PageRank from the user node; return top items not yet seen."""
    if user not in G:
        raise ValueError(f"Unknown user: {user}")
    personalization = {user: 1.0}
    pr = nx.pagerank(G, alpha=alpha, personalization=personalization)
    owned = user_items(G, user)
    items_all = [n for n, d in G.nodes(data=True) if d.get("bipartite") == "item"]
    candidates = [it for it in items_all if it not in owned]
    recs = sorted(
        ((it, pr.get(it, 0.0)) for it in candidates), key=lambda x: x[1], reverse=True
    )[:topk]
    return pd.DataFrame(recs, columns=["item", "score"])


### Compare recommenders for a sample user

Below we inspect how the different strategies behave for one user. Look for overlap between the top results and relate them back to the original interactions or item categories.


In [ ]:
demo_user = "U3"
print(f"User {demo_user} already has: {sorted(user_items(G, demo_user))}")

pop_df = recommend_popularity(df, demo_user, topk=5)
co_df = recommend_cooccurrence(G, G_item, demo_user, topk=5)
jac_df = recommend_jaccard(G, demo_user, jacc, agg="mean", topk=5)
aa_df = recommend_adamic_adar(G, demo_user, aa, agg="sum", topk=5)
ppr_df = recommend_ppr(G, demo_user, topk=5, alpha=0.85)

for name, rec_df in [
    ("Popularity", pop_df),
    ("Co-occurrence", co_df),
    ("Jaccard (mean)", jac_df),
    ("Adamic-Adar (sum)", aa_df),
    ("Personalized PageRank", ppr_df),
]:
    print(f"
{name} recommendations:")
    display(
        rec_df.merge(item_catalog, on="item", how="left")
        .assign(rank=lambda df_: range(1, len(df_) + 1))
        [["rank", "item", "category", "score"]]
    )


> **Your turn:** pick a different user (or edit their interactions) and re-run the cell above. Which strategies feel too conservative? Which ones surface more diverse content?


## Quick visualizations

Visuals help non-graph specialists understand why a recommendation appeared. We will look at degree distributions and a trimmed projection graph you can screenshot for slides.


In [ ]:
item_degs = [G.degree(i) for i in item_nodes]
plt.figure(figsize=(6, 4))
plt.hist(item_degs, bins=range(1, 1 + max(item_degs)))
plt.title("Item degree distribution")
plt.xlabel("Number of users connected")
plt.ylabel("Count of items")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
pos = {}
pos.update({u: (-1, idx) for idx, u in enumerate(sorted(user_nodes))})
pos.update({it: (1, idx) for idx, it in enumerate(sorted(item_nodes))})

nx.draw_networkx(
    G,
    pos=pos,
    node_size=[300 if n in user_nodes else 400 for n in G.nodes()],
    node_color=["#6baed6" if n in user_nodes else "#fd8d3c" for n in G.nodes()],
    with_labels=True,
    edge_color="#cccccc",
)
plt.title("User-item bipartite view")
plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
top_edges = sorted(
    G_item.edges(data=True), key=lambda e: e[2].get("weight", 0.0), reverse=True
)[:20]

H = nx.Graph()
H.add_nodes_from(item_nodes)
H.add_edges_from(
    [(a, b, {"weight": data.get("weight", 0.0)}) for a, b, data in top_edges]
)

plt.figure(figsize=(8, 6))
pos = nx.spring_layout(H, seed=42, weight="weight")
nx.draw_networkx_nodes(H, pos, node_size=400, node_color="#fd8d3c")
nx.draw_networkx_labels(H, pos)
nx.draw_networkx_edges(
    H,
    pos,
    width=[data["weight"] for _, _, data in H.edges(data=True)],
    edge_color="#999999",
)
plt.title("Top item-item co-occurrences")
plt.axis("off")
plt.tight_layout()
plt.show()


## Tiny offline evaluation (Hit-Rate@K, MRR@K, Precision@K)

We run a leave-one-out evaluation: hide the last interaction for each user, generate recommendations from the remaining history, and check where the hidden item ranks. This is not production-grade evaluation, but it is enough to compare heuristics in class.


In [ ]:
def evaluate_holdout(interactions, K=5):
    rows = []
    for u, its in interactions.items():
        if len(its) < 2:
            continue
        hidden = its[-1]
        train_its = its[:-1]
        # Build training DF (replace the user's interactions with train-only)
        rec_df = pd.DataFrame(
            [
                (uu, it, 1)
                for uu, lst in interactions.items()
                for it in (lst if uu != u else train_its)
            ],
            columns=["user", "item", "weight"],
        )
        Gtr = build_bipartite_graph(rec_df)
        user_nodes_tr = [n for n, d in Gtr.nodes(data=True) if d.get("bipartite") == "user"]
        item_nodes_tr = [n for n, d in Gtr.nodes(data=True) if d.get("bipartite") == "item"]
        Gtr_item = bipartite.weighted_projected_graph(Gtr, item_nodes_tr)

        pairs_tr = list(itertools.combinations(item_nodes_tr, 2))
        jacc_tr = {
            tuple(sorted((a, b))): s for (a, b, s) in jaccard_coefficient(Gtr, pairs_tr)
        }
        aa_tr = {
            tuple(sorted((a, b))): s for (a, b, s) in adamic_adar_index(Gtr, pairs_tr)
        }

        def rank_of(item, df_scores):
            arr = df_scores["item"].tolist()
            return (arr.index(item) + 1) if item in arr else None

        pop = recommend_popularity(rec_df, u, topk=K)
        co = recommend_cooccurrence(Gtr, Gtr_item, u, topk=K)
        jac = recommend_jaccard(Gtr, u, jacc_tr, agg="mean", topk=K)
        aad = recommend_adamic_adar(Gtr, u, aa_tr, agg="sum", topk=K)
        ppr = recommend_ppr(Gtr, u, topk=K, alpha=0.85)

        for name, df_rec in [
            ("popularity", pop),
            ("cooccurrence", co),
            ("jaccard", jac),
            ("adamic_adar", aad),
            ("ppr", ppr),
        ]:
            ranked = df_rec["item"].tolist()
            rank = rank_of(hidden, df_rec)
            hit = 1 if (rank is not None and rank <= K) else 0
            mrr = (1.0 / rank) if rank is not None else 0.0
            precision = hit / K
            rows.append(
                {
                    "user": u,
                    "held_out": hidden,
                    "method": name,
                    "rank": rank,
                    "hit@K": hit,
                    "mrr@K": mrr,
                    "precision@K": precision,
                    "recommended": tuple(ranked),
                }
            )

    results = pd.DataFrame(rows)
    summary = (
        results.groupby("method")[ ["hit@K", "mrr@K", "precision@K"] ]
        .mean()
        .reset_index()
        .sort_values("mrr@K", ascending=False)
    )
    return results, summary


results, summary = evaluate_holdout(interactions, K=5)
print("Per-user results (first 10 rows):")
display(results.head(10))

coverage = (
    results.groupby("method")["recommended"]
    .apply(lambda tuples: len(set(itertools.chain.from_iterable(tuples))))
    .reset_index(name="unique_items")
)
coverage["coverage@K"] = coverage["unique_items"] / len(item_nodes)

summary_with_cov = summary.merge(coverage, on="method")

print("Summary (averaged over users):")
display(summary_with_cov)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].bar(summary_with_cov["method"], summary_with_cov["hit@K"], color="#3182bd")
axes[0].set_title("Hit-Rate@5 by method")
axes[0].set_xlabel("Method")
axes[0].set_ylabel("Score")
axes[0].set_ylim(0, 1)
axes[0].tick_params(axis="x", rotation=20)

axes[1].bar(summary_with_cov["method"], summary_with_cov["mrr@K"], color="#9ecae1")
axes[1].set_title("MRR@5 by method")
axes[1].set_xlabel("Method")
axes[1].set_ylabel("Score")
axes[1].set_ylim(0, 1)
axes[1].tick_params(axis="x", rotation=20)

plt.tight_layout()
plt.show()

print("Coverage@5:")
display(summary_with_cov[["method", "coverage@K"]])


## Facilitation prompts

- Which metric best reflects success for your domain? Would recall or diversity matter more?
- How could we blend graph-based scores with metadata such as `category`?
- What happens if we include timestamps or weights on the edges?
- How would you productionize this flow (batch jobs, feature stores, real-time serving)?


## Next steps

- Swap in your own interactions data (CSV of `user,item[,weight]`) and re-run the exploration cells to sanity-check the degree distributions.
- Try weighting edges by rating, recency, or time spent. Observe how it shifts the co-occurrence projection.
- Add advanced algorithms such as Random Walk with Restart, node2vec embeddings, or graph neural networks once the team is comfortable with the basics.
- Export the trained graphs to `dist/` and surface them on JupyterLite so attendees can experiment after the session.
- Capture evaluation snapshots under `content/data/validation/` if you iterate on the heuristics.
